Run the following line if you don't have any of the libraries

In [3]:
%%bash
pip install statsmodels pandas numpy

Couldn't find program: 'bash'


In [9]:
import pandas as pd

import data_prep

In [12]:
data_prep.get_lagged("./data/XLE_energy_etf.csv", n=3)

,Open,High,Low,Close,Adj Close,Volume,Adj Close_1,Adj Close_2,Adj Close_3
Date,,,,,,,,,
2010-01-04,57.919998,58.810001,57.790001,58.810001,44.789688,16928400,NaN,NaN,NaN
2010-01-05,58.889999,59.410000,58.619999,59.290001,45.155247,17368100,44.789688,NaN,NaN
2010-01-06,59.320000,60.180000,59.189999,60.000000,45.695988,24351900,45.155247,44.789688,NaN
2010-01-07,59.740002,60.009998,59.299999,59.910000,45.627445,17449500,45.695988,45.155247,44.789688
2010-01-08,59.700001,60.349998,59.490002,60.299999,45.924469,13344300,45.627445,45.695988,45.155247
...,...,...,...,...,...,...,...,...,...
2020-03-16,27.950001,31.120001,27.670000,27.809999,27.809999,40233600,32.189999,29.469999,33.669998
2020-03-17,28.080000,29.180000,26.950001,28.000000,28.000000,45443600,27.809999,32.189999,29.469999
2020-03-18,26.040001,26.580000,22.879999,23.980000,23.980000,52135800,28.000000,27.809999,32.189999


In [7]:
pd.read_csv("./data/SnP_500.csv", index_col="Date").head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3991400000
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2491020000
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4972660000
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5270680000
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4389590000


In [13]:
data_prep.get_perc_return("./data/SnP_500.csv", column_name="Open")

AttributeError: module 'data_prep' has no attribute 'get_perc_return'